# Sesión 3 - Cuaderno 3: El Impacto del Z-Ordering

### **Introducción: Más Allá del Particionamiento**

En el cuaderno anterior, demostramos de manera concluyente que el **particionamiento** es una herramienta extremadamente eficaz para eliminar grandes volúmenes de datos irrelevantes a nivel de directorio. Si lo imaginamos como un archivador físico, el particionamiento es como organizar nuestros documentos en cajones separados por año. Si buscamos un informe de 2023, ignoramos por completo los cajones de 2022, 2021, etc.

Sin embargo, ¿qué pasa cuando abrimos el cajón de "2023" y necesitamos encontrar todas las facturas de un cliente específico, `id_usuario = 1025`? Si los documentos dentro del cajón están desordenados, no tenemos más remedio que revisar cada uno de ellos. Este es el límite del particionamiento.

Particionar por columnas de **alta cardinalidad** (muchos valores únicos) como `id_usuario` sería un desastre. Sería como tener un cajón para cada cliente, resultando en miles de cajones pequeños y vacíos, lo que ralentizaría enormemente el sistema (el conocido "small files problem").

Aquí es donde entra en juego nuestra segunda decisión de diseño de la Sesión 2: **Z-Ordering**.

### **¿Qué es Z-Ordering? La Organización Interna**

El Z-Ordering es una técnica que organiza los datos *dentro* de cada partición (o dentro de cada "cajón" de nuestro archivador). Agrupa los valores relacionados de las columnas especificadas (`id_usuario`, `id_producto`) para que estén físicamente juntos en los mismos archivos de datos.

Técnicamente, lo logra mapeando los valores de múltiples columnas en una única dimensión usando una [curva de orden Z](https://es.wikipedia.org/wiki/Curva_de_orden_Z), lo que tiende a mantener la localidad de los puntos de datos.

Delta Lake aprovecha esto manteniendo estadísticas (valores mínimos y máximos) para estas columnas en los metadatos de cada archivo. Cuando ejecutamos una consulta que filtra por una de estas columnas, Spark puede consultar estas estadísticas y **saltarse archivos enteros** que sabe que no contienen los datos que buscamos. Es el equivalente a tener los documentos dentro del cajón "2023" perfectamente ordenados por cliente.

**Objetivo:** En este cuaderno, vamos a:
* Comparar el rendimiento de una consulta en una tabla solo particionada frente a una tabla particionada y optimizada con Z-Ordering.
* Aprender a identificar la evidencia del **"file skipping"** (omisión de archivos) en las métricas de ejecución de una consulta.

### **Paso 1: Preparación del Entorno y las Tablas de Prueba**

Para realizar una comparación justa y científicamente válida, necesitamos aislar la variable que queremos medir: el efecto del Z-Ordering. Para ello, crearemos un entorno de prueba con dos tablas:

1.  **`pedidos_silver_partitioned`**: Nuestra tabla de **control**. Es una copia de los datos que solo está particionada por fecha.
2.  **`pedidos_silver`**: Nuestra tabla **optimizada**, que está particionada por fecha y además tiene aplicado el Z-Ordering sobre `id_usuario` y `id_producto`.

Este enfoque nos permitirá ejecutar la misma consulta en ambas tablas y atribuir cualquier diferencia en el rendimiento directamente al efecto del Z-Ordering.

In [0]:
%sql
-- Establecemos el contexto de la base de datos
USE curso_arquitecturas;

CREATE OR REPLACE TABLE pedidos_silver_partitioned
PARTITIONED BY (id_fecha)
AS SELECT * FROM pedidos_silver;

### **Paso 2: El Escenario "Antes" - Filtrado en una Tabla Solo Particionada**

Ahora, ejecutemos una consulta que filtre por un `id_usuario` específico en nuestra tabla de control, la que **no** tiene Z-Ordering. Para aislar el efecto, nos centraremos en una sola partición de fecha.

**Importante:** A diferencia del particionamiento, el beneficio del Z-Ordering **no es visible en el plan `EXPLAIN`**. El planificador de Spark sabe que va a realizar un `FileScan` en ambos casos, pero es el **motor de ejecución en tiempo real** el que utiliza las estadísticas de Z-Ordering para decidir qué archivos leer. Por lo tanto, la prueba no está en el plan, sino en las **métricas de ejecución** (cuántos archivos se leyeron realmente).

In [0]:
%sql
-- Ejecutamos la consulta para ver sus métricas de ejecución
SELECT *
FROM pedidos_silver_partitioned
WHERE id_fecha = '2025-04-13' AND id_usuario = 8368;

In [0]:
%sql
SELECT *
FROM pedidos_silver_partitioned
WHERE id_fecha = '2025-04-13';

In [0]:
%sql
SELECT *
FROM pedidos_silver
WHERE id_fecha = '2025-04-13';

In [0]:
%sql
EXPLAIN SELECT *
FROM pedidos_silver_partitioned
WHERE id_fecha = '2025-04-13';

### **Paso 4: El Escenario "Después" - Filtrado en una Tabla con Z-Ordering**

Ahora, ejecutemos la **exactamente la misma consulta**, pero esta vez sobre nuestra tabla original `pedidos_silver`, la cual fue optimizada con `ZORDER BY (id_usuario, id_producto)`.

In [0]:
%sql
-- Ejecutamos la misma consulta en la tabla optimizada
EXPLAIN SELECT *
FROM pedidos_silver -- Esta es nuestra tabla original, con Z-Ordering
WHERE id_usuario = '2025-04-13' ;

In [0]:
%sql
-- Ejecutamos la misma consulta en la tabla optimizada
SELECT *
FROM pedidos_silver -- Esta es nuestra tabla original, con Z-Ordering
WHERE id_fecha = '2025-04-13' ;

In [0]:
%sql
-- Ejecutamos la misma consulta en la tabla optimizada
SELECT *
FROM pedidos_silver -- Esta es nuestra tabla original, con Z-Ordering
WHERE id_fecha < '2025-04-14' ;

In [0]:
%sql
CREATE OR REPLACE TABLE pedidos_silver_partitioned_user
PARTITIONED BY (id_usuario)
AS SELECT * FROM pedidos_silver;

In [0]:
%sql
CREATE OR REPLACE TABLE pedidos_silver_orderer_user
AS SELECT * FROM pedidos_silver;
OPTIMIZE pedidos_silver_orderer_user ZORDER BY (id_usuario, id_pedido, id_fecha)

In [0]:
%sql
SELECT * FROM pedidos_silver_partitioned_user
WHERE id_usuario = 8369;
    


In [0]:
%sql
SELECT * FROM pedidos_silver_orderer_user
WHERE id_usuario = 8369;

In [0]:
%sql
SELECT * FROM pedidos_silver_orderer_user
WHERE id_usuario > 8368 and id_usuario < 8370;

In [0]:
%sql
SELECT * FROM pedidos_silver_orderer_user
WHERE id_usuario >= 8369;

### **Paso 5: Análisis de las Métricas (Con Z-Ordering)**

Inspecciona de nuevo las métricas de ejecución de esta segunda consulta de la misma manera que antes.

* **Resultado Esperado:** Verás que el número de **`files read` es drásticamente menor**. Siguiendo nuestro ejemplo, en lugar de leer 20 archivos, es posible que ahora solo lea 1 o 2.
* **¿Por qué? ¡Aquí está la magia del Z-Ordering en acción!**
    1.  **Partition Pruning:** Spark primero usó el filtro de partición para ir directamente al directorio `'2024-06-12'`, ignorando todos los demás días.
    2.  **File Pruning (Data Skipping):** Luego, antes de abrir cualquier archivo, consultó los metadatos de Delta Lake. Estos metadatos contienen las estadísticas (valores mínimos y máximos) de `id_usuario` para cada uno de los 20 archivos dentro de esa partición.
    3.  Rápidamente descartó todos los archivos cuyo rango de `id_usuario` no contenía el valor `2244;`. Por ejemplo, si un archivo contiene usuarios del `1000` al `1020` y otro del `1030` al `1050`, Spark sabe que no necesita abrirlos.
    4.  Finalmente, solo leyó el pequeño subconjunto de archivos que sí podían contener los datos, ahorrando una cantidad masiva de operaciones de I/O.